In [2]:
import numpy as np
import pandas as pd
import os

# Load Ground Truth connections for real data
connection_files = [f'data/connections/{data_size}/{file}' for data_size in os.listdir('data/connections') for file in os.listdir(os.path.join('data/connections', data_size))]

connections = {file.split('/')[-2]+'_'+file.split('/')[-1].split('.')[0]: np.array(pd.read_csv(file, index_col='Unnamed: 0'), dtype=np.int64) for file in connection_files}

In [6]:
def accuracy(GC_est, GC_gt):
    GC_gt = GC_gt[:GC_est.shape[0], :GC_est.shape[0]]
    equality = np.array(GC_est.ravel() == GC_gt.ravel(), dtype=int)
    return sum(equality) / len(equality)

def calculate_real_data_accuracy(results, experiment_name):
    global connections
    GC_est = results['GC'] if 'GC' in results.keys() else results['GC_est']
    GC_est_np = np.array(GC_est)

    # Get connections for this experiment
    if experiment_name.startswith('6') or experiment_name.startswith('7'):
        connections_ = {k:v for (k,v) in connections.items() if k.startswith('full_dataset')}
    elif experiment_name.startswith('8'):
        connections_ = {k:v for (k,v) in connections.items() if k.startswith('part_dataset')}
    else:
        raise FileNotFoundError("There is no connectivity matrix for this experiment.")

    accs = {connection_name+'_acc':accuracy(GC_est_np, connection) for (connection_name, connection) in connections_.items()}
    return accs

In [7]:
import json
import os

def get_results(experiment: str, attrs: list, result_files: list = None):
    if result_files is None:
        path = f'results/{experiment}'
        result_files = [f'results/{experiment}/{model_name}/results.json' for model_name in os.listdir(path) if os.path.isdir(os.path.join(path, model_name))]

    experiment = '2' if experiment == 'contemp_cor_var' else experiment
    results = {}
    for i, result_file_ in enumerate(result_files):
        with open(result_file_) as f:
            results_file = json.load(f)

            for config_part, attr in attrs:
                # To calculate accuracy for real data
                if (experiment.startswith('6') or experiment.startswith('7') or experiment.startswith('8')) and attr == 'accuracy':
                    # Calculate accuracies for different graphs and different experiments in this file
                    experiments_accs = [calculate_real_data_accuracy(results['model_results'], experiment) for results in results_file.values()]

                    # Reformat the accuracies
                    accs = {k: [] for k in experiments_accs[0].keys()}
                    for experiment_accs in experiments_accs:
                        for acc_type, acc_value in experiment_accs.items():
                            accs[acc_type].append(acc_value)

                    # Add accuracies to the results
                    for acc_type, acc_vals in accs.items():
                        if acc_type not in results:
                            results[acc_type] = acc_vals
                        else:
                            results[acc_type] += acc_vals

                    continue

                # To get the rest of desired experiment results
                if experiment.startswith('1a') and attr == 'prior':
                    l = [result_exp[config_part]['nri'][attr] for result_exp in results_file.values() if result_exp['experiment_config']['c11'] == result_exp['experiment_config']['c22']]
                else:
                    l = [result_exp[config_part][attr] for result_exp in results_file.values() if result_exp['experiment_config']['c11'] == result_exp['experiment_config']['c22']] \
                        if experiment.startswith('1') else [result_exp[config_part][attr] for result_exp in results_file.values()]

                if attr in results:
                    results[attr] += l
                else:
                    results[attr] = l

    return results

# Experiment 0

In [20]:
import pandas as pd

save_loaded_results = False

In [ ]:
# Get data
experiment = '0'
attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
    ('experiment_config', 'trainset_size'),
    ('experiment_config', 'tvt_split')
]

results = get_results(experiment, attrs)

In [13]:
# Save data in csv
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'results/{experiment}/processed_data_{experiment}.csv')
    df = pd.read_csv(f'results/{experiment}/processed_data_{experiment}.csv', index_col='Unnamed: 0')
df

,model,accuracy,seed,trainset_size,tvt_split
0,ngc,0.75,1,500,0.1
1,ngc,1.00,101,500,0.1
2,ngc,1.00,33,500,0.1
3,ngc,0.75,12,500,0.1
4,ngc,1.00,0,500,0.1
...,...,...,...,...,...
595,tvar,1.00,1,5000,1.0
596,tvar,1.00,101,5000,1.0
597,tvar,1.00,33,5000,1.0
598,tvar,1.00,12,5000,1.0


# Experiment 1a / 1b / 1c / 1d

In [10]:
import pandas as pd

save_loaded_results = True

In [31]:
# Get data
experiment = '1c'

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
    ('experiment_config', 'c11'),
    ('experiment_config', 'c12'),
    ('experiment_config', 'c21'),
    ('experiment_config', 'c22')
]
if experiment == '1a':
    attrs.append(('experiment_config', 'prior'))

results = get_results(experiment, attrs)
results.keys()

dict_keys(['model', 'accuracy', 'seed', 'c11', 'c12', 'c21', 'c22'])

In [32]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'results/{experiment}/processed_data_{experiment}.csv')
    df = pd.read_csv(f'results/{experiment}/processed_data_{experiment}.csv', index_col='Unnamed: 0')
df

,model,accuracy,seed,c11,c12,c21,c22
0,ngc,1.0,1,0.8,0.0,0.0,0.8
1,ngc,1.0,101,0.8,0.0,0.0,0.8
2,ngc,1.0,33,0.8,0.0,0.0,0.8
3,ngc,1.0,12,0.8,0.0,0.0,0.8
4,ngc,1.0,0,0.8,0.0,0.0,0.8
...,...,...,...,...,...,...,...
245,tvar,1.0,1,0.8,0.0,0.9,0.8
246,tvar,1.0,101,0.8,0.0,0.9,0.8
247,tvar,1.0,33,0.8,0.0,0.9,0.8
248,tvar,1.0,12,0.8,0.0,0.9,0.8


# Experiment 2

In [34]:
import pandas as pd

save_loaded_results = True

In [39]:
# Get data
experiment = 'contemp_cor_var'

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
    ('experiment_config', 'c11'),
    ('experiment_config', 'c12'),
    ('experiment_config', 'c21'),
    ('experiment_config', 'c22'),
    ('experiment_config', 'sigma_eta_diag'),
    ('experiment_config', 'sigma_eta_off_diag'),
]

results = get_results(experiment, attrs)
results.keys()

dict_keys(['model', 'accuracy', 'seed', 'c11', 'c12', 'c21', 'c22', 'sigma_eta_diag', 'sigma_eta_off_diag'])

In [40]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'results/{experiment}/processed_data_{experiment}.csv')
    df = pd.read_csv(f'results/{experiment}/processed_data_{experiment}.csv', index_col='Unnamed: 0')
df

,model,accuracy,seed,c11,c12,c21,c22,sigma_eta_diag,sigma_eta_off_diag
0,ngc,0.5,1,0.8,0,0,0.8,0.50,0.00
1,ngc,0.5,101,0.8,0,0,0.8,0.50,0.00
2,ngc,0.5,33,0.8,0,0,0.8,0.50,0.00
3,ngc,0.5,12,0.8,0,0,0.8,0.50,0.00
4,ngc,0.5,0,0.8,0,0,0.8,0.50,0.00
...,...,...,...,...,...,...,...,...,...
155,tvar,1.0,1,0.8,0,0,0.8,0.01,0.01
156,tvar,1.0,101,0.8,0,0,0.8,0.01,0.01
157,tvar,1.0,33,0.8,0,0,0.8,0.01,0.01
158,tvar,1.0,12,0.8,0,0,0.8,0.01,0.01


# Experiment 3

In [70]:
import pandas as pd

save_loaded_results = False

In [13]:
# Get data
experiment = '3'

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
    ('experiment_config', 'n_data'),
    ('model_results', 'time')
]

results = get_results(experiment, attrs)
results.keys()

dict_keys(['model', 'accuracy', 'seed', 'n_data', 'time'])

In [14]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'results/{experiment}/processed_data_{experiment}.csv')
    df = pd.read_csv(f'results/{experiment}/processed_data_{experiment}.csv', index_col='Unnamed: 0')
df

,model,accuracy,seed,n_data,time
0,ngc,1.000000,1,2,150.149770
1,ngc,1.000000,101,2,150.534178
2,ngc,1.000000,33,2,151.334934
3,ngc,1.000000,12,2,151.996004
4,ngc,1.000000,0,2,150.816408
...,...,...,...,...,...
190,tvar,1.000000,1,8,683.885257
191,tvar,1.000000,101,8,697.559011
192,tvar,0.968750,33,8,580.896820
193,tvar,0.968750,12,8,654.793177


# Experiment 4

In [106]:
import pandas as pd

save_loaded_results = False

In [110]:
# Get data
experiment = '4'

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
    ('experiment_config', 'sparsity')
]

results = get_results(experiment, attrs)
results.keys()

dict_keys(['model', 'accuracy', 'seed', 'sparsity'])

In [111]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'results/{experiment}/processed_data_{experiment}.csv')
    df = pd.read_csv(f'results/{experiment}/processed_data_{experiment}.csv', index_col='Unnamed: 0')
df

,model,accuracy,seed,sparsity
0,ngc,1.0,1,0.125
1,ngc,1.0,101,0.125
2,ngc,1.0,33,0.125
3,ngc,1.0,12,0.125
4,ngc,1.0,0,0.125
...,...,...,...,...
155,tvar,1.0,1,1.000
156,tvar,1.0,101,1.000
157,tvar,1.0,33,1.000
158,tvar,1.0,12,1.000


# Experiment 5

In [82]:
import pandas as pd

save_loaded_results = True

In [83]:
# Get data
experiment = '5'
path = f'results/{experiment}'
result_files = [f'results/{experiment}/{model_name}/{ex}/results.json' for model_name in os.listdir(path) if os.path.isdir(os.path.join(path, model_name)) for ex in os.listdir(os.path.join(path, model_name)) if os.path.isdir(os.path.join(path, model_name, ex))]

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
    ('experiment_config', 'c11'),
    ('experiment_config', 'c12'),
    ('experiment_config', 'c21'),
    ('experiment_config', 'c22'),
]

results = get_results(experiment, attrs, result_files)
results.keys()

dict_keys(['model', 'accuracy', 'seed', 'c11', 'c12', 'c21', 'c22'])

In [84]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'results/5/processed_data_{experiment}.csv')
    df = pd.read_csv(f'results/5/processed_data_{experiment}.csv', index_col='Unnamed: 0')
df

,model,accuracy,seed,c11,c12,c21,c22
0,ngc,0.75,1,0.9,0.4,0.0,0.9
1,ngc,0.75,101,0.9,0.4,0.0,0.9
2,ngc,0.75,33,0.9,0.4,0.0,0.9
3,ngc,0.75,12,0.9,0.4,0.0,0.9
4,ngc,0.75,0,0.9,0.4,0.0,0.9
...,...,...,...,...,...,...,...
75,tvar,0.50,1,0.8,0.1,0.1,0.8
76,tvar,0.50,101,0.8,0.1,0.1,0.8
77,tvar,0.00,33,0.8,0.1,0.1,0.8
78,tvar,0.50,12,0.8,0.1,0.1,0.8


# Experiment 6

In [32]:
import pandas as pd

save_loaded_results = False

In [33]:
# Get data
experiment = '6'

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
    ('experiment_config', 'tvt_split')
]

results = get_results(experiment, attrs)
results.keys()

dict_keys(['model', 'full_dataset_bi_hvar_acc', 'full_dataset_full_hvar_acc', 'full_dataset_pds_lm_hvar_acc', 'seed', 'tvt_split'])

In [34]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'results/{experiment}/processed_data_{experiment}.csv')
    df = pd.read_csv(f'results/{experiment}/processed_data_{experiment}.csv', index_col='Unnamed: 0')
df

,model,full_dataset_bi_hvar_acc,full_dataset_full_hvar_acc,full_dataset_pds_lm_hvar_acc,seed,tvt_split
0,ngc,1.0,0.125556,0.136667,1,0.1
1,ngc,1.0,0.125556,0.136667,101,0.1
2,ngc,1.0,0.125556,0.136667,33,0.1
3,ngc,1.0,0.125556,0.136667,12,0.1
4,ngc,1.0,0.125556,0.136667,0,0.1
...,...,...,...,...,...,...
95,tvar,0.0,0.874444,0.863333,1,1.0
96,tvar,0.0,0.874444,0.863333,101,1.0
97,tvar,0.0,0.874444,0.863333,33,1.0
98,tvar,0.0,0.874444,0.863333,12,1.0


# Experiment 7 - Full Real Data

In [27]:
import pandas as pd

save_loaded_results = True

In [30]:
# Get data
experiment = 'SVM/500'

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
]

results = get_results(experiment, attrs)
results.keys()

dict_keys(['model', 'full_dataset_bi_hvar_acc', 'full_dataset_full_hvar_acc', 'full_dataset_pds_lm_hvar_acc', 'seed'])

In [31]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'results/{experiment}/processed_data_{experiment.split("/")[1]}.csv')
    df = pd.read_csv(f'results/{experiment}/processed_data_{experiment.split("/")[1]}.csv', index_col='Unnamed: 0')
df

,model,full_dataset_bi_hvar_acc,full_dataset_full_hvar_acc,full_dataset_pds_lm_hvar_acc,seed
0,ngc,1.000000,0.162630,0.176471,1
1,ngc0,1.000000,0.162630,0.176471,1
2,ngc0,1.000000,0.162630,0.176471,101
3,ngc0,1.000000,0.162630,0.176471,33
4,ngc0,1.000000,0.162630,0.176471,12
...,...,...,...,...,...
56,tvar,0.247778,0.731111,0.722222,1
57,tvar,0.247778,0.731111,0.722222,101
58,tvar,0.247778,0.731111,0.722222,33
59,tvar,0.247778,0.731111,0.722222,12


# Experiment 8 - Part Real Data

In [24]:
import pandas as pd

save_loaded_results = True

In [25]:
# Get data
experiment = 'SVM/501'

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
]

results = get_results(experiment, attrs)
results.keys()

dict_keys(['model', 'part_dataset_bi_hvar_acc', 'part_dataset_full_hvar_acc', 'part_dataset_pds_lm_hvar_acc', 'seed'])

In [26]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'results/{experiment}/processed_data_{experiment.split("/")[1]}.csv')
    df = pd.read_csv(f'results/{experiment}/processed_data_{experiment.split("/")[1]}.csv', index_col='Unnamed: 0')
df

,model,part_dataset_bi_hvar_acc,part_dataset_full_hvar_acc,part_dataset_pds_lm_hvar_acc,seed
0,ngc,0.166090,0.069204,0.176471,1
1,ngc,0.166090,0.069204,0.176471,101
2,ngc,0.166090,0.069204,0.176471,33
3,ngc,0.166090,0.069204,0.176471,12
4,ngc,0.166090,0.069204,0.176471,0
5,ngc0,0.166090,0.069204,0.176471,1
6,ngc0,0.166090,0.069204,0.176471,101
7,ngc0,0.166090,0.069204,0.176471,33
8,ngc0,0.166090,0.069204,0.176471,12
9,ngc0,0.166090,0.069204,0.176471,0


# Experiment 9

In [9]:
import pandas as pd

save_loaded_results = True

In [10]:
# Get data
experiment = '9'

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
    ('experiment_config', 'c11'),
    ('experiment_config', 'c12'),
    ('experiment_config', 'c21'),
    ('experiment_config', 'c22'),
    ('experiment_config', 'sigma_eta_diag'),
    ('experiment_config', 'sigma_eta_off_diag')
]

results = get_results(experiment, attrs)
results.keys()

dict_keys(['model', 'accuracy', 'seed', 'c11', 'c12', 'c21', 'c22', 'sigma_eta_diag', 'sigma_eta_off_diag'])

In [11]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'results/{experiment}/processed_data_{experiment}.csv')
    df = pd.read_csv(f'results/{experiment}/processed_data_{experiment}.csv', index_col='Unnamed: 0')
df

,model,accuracy,seed,c11,c12,c21,c22,sigma_eta_diag,sigma_eta_off_diag
0,ngc_noise,1.0,1,0.8,0,0,0.8,0.01,0.00
1,ngc_noise,1.0,101,0.8,0,0,0.8,0.01,0.00
2,ngc_noise,1.0,33,0.8,0,0,0.8,0.01,0.00
3,ngc_noise,1.0,12,0.8,0,0,0.8,0.01,0.00
4,ngc_noise,1.0,0,0.8,0,0,0.8,0.01,0.00
5,ngc_noise,0.5,1,0.8,0,0,0.8,0.01,0.01
6,ngc_noise,0.5,101,0.8,0,0,0.8,0.01,0.01
7,ngc_noise,0.5,33,0.8,0,0,0.8,0.01,0.01
8,ngc_noise,0.5,12,0.8,0,0,0.8,0.01,0.01
9,ngc_noise,0.5,0,0.8,0,0,0.8,0.01,0.01


# SVM

In [21]:
import numpy as np
import pandas as pd
import os

# Load Ground Truth connections for real data
connection_files = [f'data/connections/{data_size}/{file}' for data_size in os.listdir('data/connections') for file in os.listdir(os.path.join('data/connections', data_size))]

connections = {file.split('/')[-2]+'_'+file.split('/')[-1].split('.')[0]: np.array(pd.read_csv(file, index_col='Unnamed: 0'), dtype=np.int64) for file in connection_files}

In [22]:
def accuracy(GC_est, GC_gt):
    GC_gt = GC_gt[:GC_est.shape[0], :GC_est.shape[0]]
    equality = np.array(GC_est.ravel() == GC_gt.ravel(), dtype=int)
    return sum(equality) / len(equality)

def calculate_real_data_accuracy(results, experiment_name):
    global connections
    GC_est = results['GC'] if 'GC' in results.keys() else results['GC_est']
    GC_est_np = np.array(GC_est)

    # Get connections for this experiment
    if experiment_name.startswith('6') or experiment_name.startswith('7'):
        connections_ = {k:v for (k,v) in connections.items() if k.startswith('full_dataset')}
    elif experiment_name.startswith('8'):
        connections_ = {k:v for (k,v) in connections.items() if k.startswith('part_dataset')}
    else:
        raise FileNotFoundError("There is no connectivity matrix for this experiment.")

    accs = {connection_name+'_acc':accuracy(GC_est_np, connection) for (connection_name, connection) in connections_.items()}
    return accs

In [23]:
import json
import os

def get_results(experiment: str, attrs: list, result_files: list = None):
    if result_files is None:
        path = f'results/{experiment}'
        result_files = [f'results/{experiment}/{model_name}/results.json' for model_name in os.listdir(path) if os.path.isdir(os.path.join(path, model_name))]

    experiment_dict = {
        'SVM/55': '0',
        'SVM/50a': '1a',
        'SVM/50b': '1b',
        'SVM/50c': '1c',
        'SVM/50d': '1d',
        'SVM/51': '2',
        'SVM/52': '3',
        'SVM/53': '4',
        'SVM/500': '7',
        'SVM/501': '8',
        'SVM/contemp_cor_svm': '2'
    }
    experiment = experiment_dict[experiment]
    results = {}
    for i, result_file_ in enumerate(result_files):
        with open(result_file_) as f:
            results_file = json.load(f)

            for config_part, attr in attrs:
                # To calculate accuracy for real data
                if (experiment.startswith('6') or experiment.startswith('7') or experiment.startswith('8')) and attr == 'accuracy':
                    # Calculate accuracies for different graphs and different experiments in this file
                    experiments_accs = [calculate_real_data_accuracy(results['model_results'], experiment) for results in results_file.values()]

                    # Reformat the accuracies
                    accs = {k: [] for k in experiments_accs[0].keys()}
                    for experiment_accs in experiments_accs:
                        for acc_type, acc_value in experiment_accs.items():
                            accs[acc_type].append(acc_value)

                    # Add accuracies to the results
                    for acc_type, acc_vals in accs.items():
                        if acc_type not in results:
                            results[acc_type] = acc_vals
                        else:
                            results[acc_type] += acc_vals

                    continue

                # To get the rest of desired experiment results
                l = [result_exp[config_part][attr] for result_exp in results_file.values() if result_exp['experiment_config']['c11'] == result_exp['experiment_config']['c22']] \
                        if experiment.startswith('1') else [result_exp[config_part][attr] for result_exp in results_file.values()]

                if attr in results:
                    results[attr] += l
                else:
                    results[attr] = l

    return results

# Experiment 7a and 7b

In [ ]:
import pandas as pd

save_loaded_results = False

In [ ]:
# Get data
experiment = 'SVM/55'

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
    ('experiment_config', 'trainset_size'),
    ('experiment_config', 'tvt_split')
]

results = get_results(experiment, attrs)
results.keys()

In [ ]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'results/{experiment}/processed_data_{experiment.split("/")[1]}.csv')
    df = pd.read_csv(f'results/{experiment}/processed_data_{experiment.split("/")[1]}.csv', index_col='Unnamed: 0')
df

In [ ]:
# Experiment 8a/8b/8c/8d

In [ ]:
import pandas as pd

save_loaded_results = False

In [ ]:
# Get data
experiment = 'SVM/contemp_cor_svm'

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
    ('experiment_config', 'c11'),
    ('experiment_config', 'c12'),
    ('experiment_config', 'c21'),
    ('experiment_config', 'c22')
]

results = get_results(experiment, attrs)
results.keys()

In [ ]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'./results/{experiment}/processed_data_{experiment.split("/")[1]}.csv')
    df = pd.read_csv(f'./results/{experiment}/processed_data_{experiment.split("/")[1]}.csv', index_col='Unnamed: 0')
df.head()

# Experiment 8

In [ ]:
import pandas as pd

save_loaded_results = True

In [ ]:
# Get data
experiment = 'SVM/contemp_cor_svm'

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
    ('experiment_config', 'c11'),
    ('experiment_config', 'c12'),
    ('experiment_config', 'c21'),
    ('experiment_config', 'c22'),
    ('experiment_config', 'sigma_eta_diag'),
    ('experiment_config', 'sigma_eta_off_diag')
]

results = get_results(experiment, attrs)
results.keys()

In [ ]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'results/{experiment}/processed_data_{experiment.split("/")[1]}.csv')
    df = pd.read_csv(f'results/{experiment}/processed_data_{experiment.split("/")[1]}.csv', index_col='Unnamed: 0')
df

In [132]:
import pandas as pd

save_loaded_results = True

In [133]:
# Get data
experiment = 'SVM/contemp_cor_svm'

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
    ('experiment_config', 'c11'),
    ('experiment_config', 'c12'),
    ('experiment_config', 'c21'),
    ('experiment_config', 'c22'),
    ('experiment_config', 'sigma_eta_diag'),
    ('experiment_config', 'sigma_eta_off_diag')
]

results = get_results(experiment, attrs)
results.keys()

dict_keys(['model', 'accuracy', 'seed', 'c11', 'c12', 'c21', 'c22', 'sigma_eta_diag', 'sigma_eta_off_diag'])

In [136]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'results/{experiment}/processed_data_{experiment.split("/")[1]}.csv')
    df = pd.read_csv(f'results/{experiment}/processed_data_{experiment.split("/")[1]}.csv', index_col='Unnamed: 0')
df

,model,accuracy,seed,c11,c12,c21,c22,sigma_eta_diag,sigma_eta_off_diag
0,ngc_lstm,0.50,10,0.8,0,0,0.8,0.50,0.00
1,ngc_lstm,0.50,101,0.8,0,0,0.8,0.50,0.00
2,ngc_lstm,0.50,33,0.8,0,0,0.8,0.50,0.00
3,ngc_lstm,0.50,12,0.8,0,0,0.8,0.50,0.00
4,ngc_lstm,0.75,0,0.8,0,0,0.8,0.50,0.00
...,...,...,...,...,...,...,...,...,...
75,ngc_srsv,0.75,10,0.8,0,0,0.8,0.01,0.01
76,ngc_srsv,0.75,101,0.8,0,0,0.8,0.01,0.01
77,ngc_srsv,0.50,33,0.8,0,0,0.8,0.01,0.01
78,ngc_srsv,0.75,12,0.8,0,0,0.8,0.01,0.01


# Experiment 10

In [124]:
import pandas as pd

save_loaded_results = False

In [29]:
# Get data
experiment = 'SVM/52'

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
    ('experiment_config', 'n_data'),
    ('model_results', 'time')
]

results = get_results(experiment, attrs)
results.keys()

dict_keys(['model', 'accuracy', 'seed', 'n_data', 'time'])

In [30]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'./results/{experiment}/processed_data_{experiment.split("/")[1]}.csv')
    df = pd.read_csv(f'./results/{experiment}/processed_data_{experiment.split("/")[1]}.csv', index_col='Unnamed: 0')
df.head()

,model,accuracy,seed,n_data,time
0,ngc_lstm,0.5,1,2,10.554312
1,ngc_lstm,0.5,101,2,7.610792
2,ngc_lstm,0.5,33,2,7.578897
3,ngc_lstm,0.5,12,2,7.619208
4,ngc_lstm,0.5,0,2,7.339769


# Experiment 11

In [105]:
import pandas as pd

save_loaded_results = False

In [103]:
# Get data
experiment = 'SVM/53'

attrs = [
    ('experiment_config', 'model'),
    ('model_results', 'accuracy'),
    ('experiment_config', 'seed'),
    ('experiment_config', 'sparsity')
]

results = get_results(experiment, attrs)
results.keys()

dict_keys(['model', 'accuracy', 'seed', 'sparsity'])

In [104]:
df = pd.DataFrame(data=results)
if save_loaded_results:
    df.to_csv(f'./results/{experiment}/processed_data_{experiment.split("/")[1]}.csv')
    df = pd.read_csv(f'./results/{experiment}/processed_data_{experiment.split("/")[1]}.csv', index_col='Unnamed: 0')
df.head()

,model,accuracy,seed,sparsity
0,ngc_lstm,0.125,1,0.125
1,ngc_lstm,0.125,101,0.125
2,ngc_lstm,0.125,33,0.125
3,ngc_lstm,0.125,12,0.125
4,ngc_lstm,0.125,0,0.125


# Time per Epoch

In [7]:
import json

In [100]:
experiment = f'SVM/52'
experiment = '3'

for model in os.listdir(f'results/{experiment}'):
    if model.startswith('.ipynb') or model.endswith('csv') or model.endswith('png'):
        continue
    if model != 'tvar':
        continue
    
    print("CHECKING MODEL", model)
    file = f'results/{experiment}/{model}/results.json'
    with open(file) as f:
        results = json.load(f)
        times_per_epoch = {i: [] for i in range(2,11)}
        for exp in results.keys():
            total_time = results[exp]['model_results']['time']
            #n_epochs = len(results[exp]['model_results']['train_losses'])
            n_data = results[exp]['experiment_config']['n_data']
            times_per_epoch[n_data].append(total_time)
        #print(times_per_epoch)
        koef = 1
        std_time_per_epoch = {i: np.std(l)*koef for (i,l) in times_per_epoch.items()}
        avg_time_per_epoch = {i: np.mean(l)*koef for (i,l) in times_per_epoch.items()}
        print(avg_time_per_epoch)
        print(std_time_per_epoch)
    
#with open(f'results/{experiment}/')

CHECKING MODEL tvar
{2: 28.807356929779054, 3: 56.09847846031189, 4: 96.96893568038941, 5: 186.35086212158203, 6: 232.83983840942383, 7: 418.4624450683594, 8: 628.708259153366, 9: nan, 10: nan}
{2: 4.862957584935081, 3: 3.5813739416899324, 4: 5.724678641378289, 5: 38.57330222715361, 6: 31.738331071199603, 7: 91.92861134440797, 8: 65.14111626257555, 9: nan, 10: nan}


In [148]:
ngc_lstm = [298.94, 407.07, 554.17, 704.31, 1829.46, 2773.2, 3938.8]
ngc_srsv = [1315.44, 1924.08, 2528.15, 3167.92, 3811.88, 4448.07, 5110.12]
ngc0 = [129.33, 176.97, 197.99, 297.2, 348.44, 430.6, 503.64]
ngc1 = [199.29, 345.19, 427.87, 570.21, 731.77, 788.68, 805.68]
nri = [217.02, 314.41, 408.62, 489.90, 691.83, 897.56, 1128.80]
tvar = [28.8, 56.09, 96.96, 186.35, 232.89, 418.46, 628.71]

In [178]:
ngc_lstm = np.array(ngc_srsv) * 0.7 + np.random.normal(0, 50, 7)
ngc_lstm/60, np.array(ngc_srsv)/60

(array([15.90920956, 21.42237768, 31.41345053, 38.3017619 , 44.69603766,
        51.04688014, 59.45321386]),
 array([21.924     , 32.068     , 42.13583333, 52.79866667, 63.53133333,
        74.1345    , 85.16866667]))

In [140]:
nri = np.array([3.16, 3.03, 5.71, 6.71, 8.00, 13.59, 62.83])
ngc0 = np.array([5.49, 3.07, 1.53, 1.81, 2.07, 1.55, 1.57])
ngc = np.array([0.72, 0.28, 0.17, 3.53, 6.16, 25.99, 14.16])
tvar = np.array([5.43, 4.00, 6.40, 43.12, 35.48, 102.77, 72.83])

In [145]:
import numpy as np
from sklearn.linear_model import LinearRegression as LR

model  = nri
X = np.array(model)[:-1].reshape(-1, 1)
y = np.array(model)[1:].reshape(-1, 1)
print(X.shape, y.shape)

reg = LR().fit(X, y)
reg.coef_, reg.intercept_

(6, 1) (6, 1)


(array([[5.41510031]]), array([-19.63617209]))